In [1]:
import pandas as pd

In [32]:
ridership_df = pd.read_csv('CTA_ridership_daily_totals.csv')

In [11]:
stops["MAP_ID"].value_counts()

41220    4
40900    4
41320    4
40540    4
40380    4
        ..
40610    2
41120    2
40090    2
40600    2
40440    2
Name: MAP_ID, Length: 143, dtype: int64

In [33]:
l_stops_df = pd.read_csv('CTA_list_of_L_stops.csv')
station_bools = l_stops_df[['MAP_ID','ADA','RED','BLUE','G','BRN','P','Pexp','Y','Pnk','O']].groupby('MAP_ID').any()
l_stations_df = l_stops_df[['MAP_ID','STATION_NAME','STATION_DESCRIPTIVE_NAME','Location']] \
    .merge(station_bools, how='left', left_on='MAP_ID', right_index=True).drop_duplicates()

In [34]:
l_stations_df[['latitude','longitude']] = \
   l_stations_df['Location'].str.replace('\(|\)','',regex=True) \
   .str.split(',', expand=True).apply(pd.to_numeric)
l_stations_df.drop('Location', axis=1, inplace=True)


In [37]:
df = pd.merge(ridership_df, l_stations_df, how='left', left_on='station_id', right_on='MAP_ID')
df[df['latitude'] < 41.881]['rides']

2          722
4          764
9          586
18         441
22        1190
          ... 
258089     394
258093    2449
258096    1168
258101     486
258107     373
Name: rides, Length: 109556, dtype: int64

In [41]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

# Path to sqlite, THIS MAY NOT MATCH YOUR PATH
database_path = "Census_Data.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [42]:
inspector = inspect(engine)
inspector.get_table_names()

['Census_Data']

In [43]:
inspector.get_columns("Census_Data")[0]

{'name': 'CityState',
 'type': TEXT(),
 'nullable': True,
 'default': None,
 'autoincrement': 'auto',
 'primary_key': 0}

In [44]:
column_names = inspector.get_columns("Census_Data")
for column in column_names:
    print(column["name"])

CityState
city
state
Population
White Population
Black Population
Native American Population
Asian Population
Hispanic Population
Education None
Education High School
Education GED
Education Associates
Education Bachelors
Education Masters
Education Professional
Education Doctorate
Poverty
Employment Labor Force
Employment Unemployed
Employment Male Computer Engineering
Employment Female Computer Engineering
Median Age
Median Male Age
Median Female Age
Household Income
Income Per Capita
Median Gross Rent
Median Home Value
lat
lng


In [56]:
data = engine.execute("SELECT CityState, [Median Age] FROM Census_Data  WHERE Population >= 100000 ORDER BY [Median Age]")
for record in data:
    print(record)

('PROVO, UT', 23.83333333)
('ATHENS, GA', 25.31666667)
('BLOOMINGTON, IN', 25.88333333)
('COLLEGE STATION, TX', 25.9)
('COLUMBIA, MO', 26.925)
('SAN BERNARDINO, CA', 27.7)
('MISSION, TX', 27.96666667)
('FARGO, ND', 28.1)
('GREENVILLE, NC', 28.1)
('PERRIS, CA', 28.15)
('BELL GARDENS, CA', 28.2)
('EDINBURG, TX', 28.3)
('LAREDO, TX', 28.66666667)
('NEWARK, DE', 28.92)
('COMPTON, CA', 28.96666667)
('MERCED, CA', 29.03333333)
('KILLEEN, TX', 29.25)
('WACO, TX', 29.33333333)
('CHARLESTON, SC', 29.34)
('LUBBOCK, TX', 29.4)
('ANN ARBOR, MI', 29.48)
('PROVIDENCE, RI', 29.7)
('FONTANA, CA', 29.86666667)
('VICTORVILLE, CA', 30.0)
('NORFOLK, VA', 30.12142857)
('NEW HAVEN, CT', 30.14)
('MORENO VALLEY, CA', 30.15)
('BROWNSVILLE, TX', 30.16666667)
('ABILENE, TX', 30.16666667)
('ODESSA, TX', 30.26666667)
('RIALTO, CA', 30.35)
('COLUMBIA, SC', 30.38571429)
('POMONA, CA', 30.43333333)
('GAINESVILLE, FL', 30.54)
('SANTA CLARA, CA', 30.625)
('MURFREESBORO, TN', 30.7)
('HARTFORD, CT', 30.75)
('ROCK HILL, S

In [90]:
import pandas as pd
import re

dracula_df = pd.read_csv('dracula.txt', sep='\n', header=None)
dracula_df.columns = ['text']

p = 'vampire'
dracula_df['text'].str.extractall(p)  #[0].value_counts()

ValueError: pattern contains no capture groups

In [75]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify

engine = create_engine("sqlite:///titanic.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

Passenger = Base.classes.passenger

app = Flask(__name__)

@app.route("/")
def welcome():
   """List all available api routes."""
   return (
       f"Available Routes:<br/>"
       f"/api/v1.0/names<br/>"
       f"/api/v1.0/passengers"
   )


if __name__ == '__main__':
   app.run(debug=True)

AttributeError: passenger